In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import argparse
import warnings
import logging
from os.path import join
from collections import defaultdict
sys.path.append('../')
warnings.filterwarnings("ignore")

from coffea import processor, util
import hist
from matplotlib import pyplot as plt
from cycler import cycler
from hist import Hist
from hist.intervals import ratio_uncertainty
from azh_analysis.utils.plotting import plot_data_vs_mc
from azh_analysis.utils.parameters import get_lumis
import warnings
warnings.filterwarnings('ignore')
#hep.style.use(["CMS", "fira", "firamath"])

mass = "300" 
years = ["2018", "2016preVFP", "2016postVFP", "2017"] # "2016preVFP", "2016postVFP", "2017", "2018"
year_label = years[0] if len(years)==1 else "all-years"
if len(years)==2: year_label = "2016"
lumis = get_lumis()
lumi = sum([lumis[year] for year in years])
if len(years)==4: lumi=lumis["Run 2"]
if len(years)==2: lumi=lumis["2016"]

rebin = True
indir = "output_unblind"
variables = ["m4l", "mll", "mtt", "pt", "met", "m4l_binopt", "met_phi"]
data, data_ss, mc, ggA, bbA = {}, {}, {}, {}, {}
for var in variables:
    data[var] = sum([sum(util.load(f"../{indir}/data_UL_{year}_OS_ub.coffea")[var].values()) for year in years])
    data_ss[var] = sum([sum(util.load(f'../{indir}/data_UL_{year}_SS_ub_relaxed.coffea')[var].values()) for year in years])
    mc[var] = sum([sum(util.load(f"../{indir}/MC_UL_{year}_all_OS.coffea")[var].values()) for year in years])
    ggA[var] = sum([
        sum([v for k, v in util.load(f"../{indir}/signal_UL_{year}_all_OS.coffea")[var].items() if (mass in k) and ("GluGlu" in k)]) 
        for year in years
    ])
    bbA[var] = sum([
        sum([v for k, v in util.load(f"../{indir}/signal_UL_{year}_all_OS.coffea")[var].items() if (mass in k) and ("GluGlu" not in k)]) 
        for year in years
    ])
    
cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

plot = True
if len(years)==4:
    outdir = f"../plots/data-mc/all"
elif len(years)==2 and "2016postVFP" in years:
    outdir = f"../plots/data-mc/2016"
elif len(years)==1:
    outdir = f"../plots/data-mc/{years[0]}"
else:
    year_str = years[0]
    for year in years[1:]: year_str = year_str + f"_{year}"
    outdir = f"../plots/data-mc/{year_str}"
print(outdir)    
print(mc.keys())

In [ ]:
(data["m4l_binopt"]["data", "eeet", ::sum, ::sum, "cons", :] + 
 data["m4l_binopt"]["data", "mmet", ::sum, ::sum, "cons", :]).data

In [ ]:
import numpy as np

def rebin_hist(h, axis_name, edges):
    if type(edges) == int:
        return h[{axis_name : hist.rebin(edges)}]

    ax = h.axes[axis_name]
    ax_idx = [a.name for a in h.axes].index(axis_name)
    if not all([np.isclose(x, ax.edges).any() for x in edges]):
        raise ValueError(f"Cannot rebin histogram due to incompatible edges for axis '{ax.name}'\n"
                            f"Edges of histogram are {ax.edges}, requested rebinning to {edges}")
        
    # If you rebin to a subset of initial range, keep the overflow and underflow
    overflow = ax.traits.overflow or (edges[-1] < ax.edges[-1] and not np.isclose(edges[-1], ax.edges[-1]))
    underflow = ax.traits.underflow or (edges[0] > ax.edges[0] and not np.isclose(edges[0], ax.edges[0]))
    flow = overflow or underflow
    new_ax = hist.axis.Variable(edges, name=ax.name, overflow=overflow, underflow=underflow)
    axes = list(h.axes)
    axes[ax_idx] = new_ax
    
    hnew = hist.Hist(*axes, name=h.name, storage=h._storage_type())

    # Offset from bin edge to avoid numeric issues
    offset = 0.5*np.min(ax.edges[1:]-ax.edges[:-1])
    edges_eval = edges+offset
    edge_idx = ax.index(edges_eval)
    # Avoid going outside the range, reduceat will add the last index anyway
    if edge_idx[-1] == ax.size+ax.traits.overflow:
        edge_idx = edge_idx[:-1]

    if underflow:
        # Only if the original axis had an underflow should you offset
        if ax.traits.underflow:
            edge_idx += 1
        edge_idx = np.insert(edge_idx, 0, 0)

    # Take is used because reduceat sums i:len(array) for the last entry, in the case
    # where the final bin isn't the same between the initial and rebinned histogram, you
    # want to drop this value. Add tolerance of 1/2 min bin width to avoid numeric issues
    hnew.values(flow=flow)[...] = np.add.reduceat(h.values(flow=flow), edge_idx, 
            axis=ax_idx).take(indices=range(new_ax.size+underflow+overflow), axis=ax_idx)
    if hnew._storage_type() == hist.storage.Weight():
        hnew.variances(flow=flow)[...] = np.add.reduceat(h.variances(flow=flow), edge_idx, 
                axis=ax_idx).take(indices=range(new_ax.size+underflow+overflow), axis=ax_idx)
    return hnew

edges = (
    np.arange(200, 500, 20).tolist() +
    np.arange(500, 750, 25).tolist() +
    np.arange(750, 1150, 100).tolist()
)

In [ ]:
########
## m4l #
########

var = "m4l_binopt"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {0: "0-btags", 1: "btags"}.items():
        for mass_type, mass_label in {"raw": "Visible", "corr": "Corrected", "cons": "Constrained"}.items():             
            mc_s, data_s, data_ss_s, ggA_s, bbA_s = mc[var], data[var], data_ss[var], ggA[var], bbA[var]
            mc_s = (
                mc_s[:, "ee" + cat, bcat, "nom", mass_type, :] +
                mc_s[:, "mm" + cat, bcat, "nom", mass_type, :]
            )
            if rebin: mc_s = rebin_hist(mc_s, "mass", edges)
            data_s = (
                data_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            if rebin: data_s = rebin_hist(data_s, "mass", edges)
            data_ss_s = (
                data_ss_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_ss_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            if rebin: data_ss_s = rebin_hist(data_ss_s, "mass", edges)
            ggA_s = (
                ggA_s[:, "ee"+cat, bcat, "nom", mass_type, :] + 
                ggA_s[:, "mm"+cat, bcat, "nom", mass_type, :]
            )
            if rebin: ggA_s = rebin_hist(ggA_s, "mass", edges)
            bbA_s = (
                bbA_s[:, "ee"+cat, bcat, "nom", mass_type, :] + 
                bbA_s[:, "mm"+cat, bcat, "nom", mass_type, :]
            )
            if rebin: bbA_s = rebin_hist(bbA_s, "mass", edges)
            var_labels = {
                "corr": r"$m_{ll\tau\tau}^\mathrm{corr}$ [GeV]",
                "cons": r"$m_{ll\tau\tau}^\mathrm{cons}$ [GeV]",
                "raw": r"$m_{ll\tau\tau}^\mathrm{raw}$ [GeV]",
            }
            name = f"{cat}_{bcat}_m4l_{mass_type}"
            plot_data_vs_mc(
                data_s, mc_s, "mass", 
                cat_label=cat_label, var_label=var_labels[mass_type], btag_label=btag_label,
                outfile=join(outdir, name+".pdf") if plot else None,
                #join(
                    #"for_alexis/towards_unblinding/plots",
                    #f"{year_label}_{cat}_{bcat}_m4l_{mass_type}.pdf") if plot else None,
                logscale=True,
                year=None,
                lumi=lumi,
                ggA=ggA_s, ggA_sigma=5, ggA_mass=mass,
                bbA=bbA_s, bbA_sigma=5, bbA_mass=mass,
                ylim=[0.01, 30],
                data_ss=data_ss_s,
                blind=True,
                blind_range=None,
                is_SR=True,
                rootfile=join("for_alexis/towards_unblinding/control_cards", name+".root"),
           )

In [ ]:
#########
## mll ##
#########

var = "mll"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in [("all", "all-btags")]: #{0: "0-btags", 1: "btags"}.items():
        mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
        mc_s = (
            mc_s[:, "ee" + cat, ::sum, "nom", :] + 
            mc_s[:, "mm" + cat, ::sum, "nom", :]
        )
        data_s = (
            data_s[:, "ee" + cat, ::sum, "none", :] + 
            data_s[:, "mm" + cat, ::sum, "none", :]
        )
        data_ss_s = (
            data_ss_s[:, "ee" + cat, ::sum, "none", :] + 
            data_ss_s[:, "mm" + cat, ::sum, "none", :]
        )
        name = f"{cat}_{bcat}_mll"
        plot_data_vs_mc(
            data_s, 
            mc_s, 
            "mll", cat_label=cat_label, var_label=r"$m_{ll}$ [GeV]", btag_label=btag_label,
            outfile=join(outdir, name+".pdf") if plot else None,
            #outfile=join(
            #    "for_alexis/towards_unblinding/plots",
            #    f"{year_label}_{cat}_{bcat}_{var}.pdf") if plot else None,
            lumi=lumi,
            year=None,
            data_ss=data_ss_s,
            blind=False,
            blind_range=None,
        )

In [ ]:
#########
## mtt ##
#########

var = "mtt"
for cat, cat_label in cat_labels.items():
    for mass_type, mass_label in {"raw": "Visible", "corr": "Corrected", "cons": "Constrained"}.items(): 
        blind_ranges = {"raw": [3,7], "corr": [3,8], "cons": [7, 12]}
        mc_s, data_s, data_ss_s, ggA_s = mc[var], data[var], data_ss[var], ggA[var]
        mc_s = (
            mc_s[:, "ee" + cat, ::sum, "nom", mass_type, :] + 
            mc_s[:, "mm" + cat, ::sum, "nom", mass_type, :]
        )
        data_s = (
            data_s[:, "ee" + cat, ::sum, "none", mass_type, :] + 
            data_s[:, "mm" + cat, ::sum, "none", mass_type, :]
        )
        data_ss_s = (
            data_ss_s[:, "ee" + cat, ::sum, "none", mass_type, :] + 
            data_ss_s[:, "mm" + cat, ::sum, "none", mass_type, :]
        )
        ggA_s = (
            ggA_s[:, "ee" + cat, ::sum, "nom", mass_type, :] + 
            ggA_s[:, "mm" + cat, ::sum, "nom", mass_type, :]
        )
        name = f"{cat}_{bcat}_mtt_{mass_type}"
        plot_data_vs_mc(
            data_s, 
            mc_s, 
            "mtt", 
            cat_label=cat_label, var_label=f"{mass_label}" + r" $m_{tt}$ [GeV]", btag_label="all btags",
            outfile=join(outdir, name+".pdf") if plot else None,
            #outfile=join(
            #    "for_alexis/towards_unblinding/plots",
            #    f"{year_label}_{cat}_all_{var}-{mass_type}.pdf") if plot else None,
            lumi=lumi,
            year=None,
            data_ss=data_ss_s,
            blind=False,
            blind_range=None, #blind_ranges[mass_type],
            ggA=ggA_s,
            ggA_sigma=5,
            ggA_mass=400,
        )

In [ ]:
#########
## MET ##
#########

var = "met"
for cat, cat_label in cat_labels.items():
    mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
    mc_s = (
        mc_s[:, "ee" + cat, ::sum, "nom", :] + 
        mc_s[:, "mm" + cat, ::sum, "nom", :]
    )
    data_s = (
        data_s[:, "ee" + cat, ::sum, "none", :] + 
        data_s[:, "mm" + cat, ::sum, "none", :]
    )
    data_ss_s = (
        data_ss_s[:, "ee" + cat, ::sum, "none", :] + 
        data_ss_s[:, "mm" + cat, ::sum, "none", :]
    )
    name = f"{cat}_{bcat}_met"
    plot_data_vs_mc(
        data_s, 
        mc_s, 
        "met", cat_label=cat_label, var_label=r"$E_T^\mathrm{miss}$", btag_label="all btags",
        outfile=join(outdir, name+".pdf") if plot else None,
        #outfile=join(
        #    "for_alexis/towards_unblinding/plots",
        #    f"{year_label}_{cat}_all_{var}.pdf") if plot else None,
        lumi=lumi,
        year=None,
        data_ss=data_ss_s,
        blind=False,
        blind_range=None#[1,4],
    )

In [ ]:
########
## pt ##
########

var = "pt"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {"all": "all"}.items():
        for leg in ['1', '2', '3', '4']:
            mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
            mc_s = (
                mc_s[:, "ee" + cat, ::sum, "nom", leg, :] + 
                mc_s[:, "mm" + cat, ::sum, "nom", leg, :]
            )
            data_s = (
                data_s[:, "ee" + cat, ::sum, "none", leg, :] + 
                data_s[:, "mm" + cat, ::sum, "none", leg, :]
            )
            data_ss_s = (
                data_ss_s[:, "ee" + cat, ::sum, "none", leg, :] + 
                data_ss_s[:, "mm" + cat, ::sum, "none", leg, :]
            )
            name = f"{cat}_{bcat}_pt_{leg}"
            plot_data_vs_mc(
                data_s, 
                mc_s, 
                "pt", cat_label=cat_label, var_label=f"Leg {leg} " + r"$p_T$", btag_label=btag_label,
                outfile=join(outdir, name+".pdf") if plot else None,
                #outfile=join(
                #    "for_alexis/towards_unblinding/plots",
                #    f"{year_label}_{cat}_all_{var}-{leg}.pdf") if plot else None,
                lumi=lumi,
                year=None,
                data_ss=data_ss_s,
                blind=False,
                blind_range=None,
            )

In [ ]:
var = "met_phi"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {"all": "all"}.items():
        mc_s, data_s, data_ss_s = mc[var], data[var], data_ss[var]
        mc_s = (
            mc_s[:, "ee" + cat, ::sum, "nom", :] + 
            mc_s[:, "mm" + cat, ::sum, "nom", :]
        )
        data_s = (
            data_s[:, "ee" + cat, ::sum, "none", :] + 
            data_s[:, "mm" + cat, ::sum, "none", :]
        )
        data_ss_s = (
            data_ss_s[:, "ee" + cat, ::sum, "none", :] + 
            data_ss_s[:, "mm" + cat, ::sum, "none", :]
        )
        name = f"{cat}_{bcat}_met_phi"
        plot_data_vs_mc(
            data_s, 
            mc_s, 
            "met_phi", cat_label=cat_label, var_label=r"MET $\phi$", btag_label=btag_label,
            outfile=join(outdir, name+".pdf") if plot else None,
            #outfile=join(
            #    "for_alexis/towards_unblinding/plots",
            #    f"{year_label}_{cat}_all_{var}.pdf") if plot else None,
            lumi=lumi,
            year=None,
            data_ss=data_ss_s,
            blind=False,
            blind_range=None,
        )

## Same-Sign Cross-Check

In [ ]:
#######################
## same-sign control ##
#######################

mass = "300" 
years = ["2016preVFP", "2016postVFP", "2017", "2018"] 
year_label = years[0] if len(years)==1 else "all-years"
if len(years)==2: year_label = "2016"
lumis = get_lumis()
lumi = sum([lumis[year] for year in years])
if len(years)==4: lumi=lumis["Run 2"]
if len(years)==2: lumi=lumis["2016"]

indir = "output_high-stats-SS-redo"
variables = ["m4l", "mll", "mtt", "pt", "met", "m4l_binopt"]
data_ss, data_ss_r, mc = {}, {}, {}
for var in variables:
    data_ss_r[var] = sum([sum(util.load(f'../{indir}/data_UL_{year}_SS_ub_relaxed.coffea')[var].values()) for year in years])
    data_ss[var] = sum([sum(util.load(f'../{indir}/data_UL_{year}_SS_ub_not-relaxed.coffea')[var].values()) for year in years])
    mc[var] = sum([sum(util.load(f"../{indir}/MC_UL_{year}_None_SS.coffea")[var].values()) for year in years])

cat_labels = {
    'tt': r'$ll\tau_h\tau_h$',
    'et': r'$ll e\tau_h$',
    'mt': r'$ll\mu\tau_h$',
    'em': r'$ll e\mu$',
}

plot = True
if len(years)==4:
    outdir = f"../plots/data-mc/all"
elif len(years)==2 and "2016postVFP" in years:
    outdir = f"../plots/data-mc/2016"
elif len(years)==1:
    outdir = f"../plots/data-mc/{years[0]}"
else:
    year_str = years[0]
    for year in years[1:]: year_str = year_str + f"_{year}"
    outdir = f"../plots/data-mc/{year_str}"
    
outdir = "for_alexis/high-stats-SS-redo"

In [ ]:
########
## m4l #
########
from azh_analysis.utils.plotting import plot_data_vs_mc

var = "m4l_binopt"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in {0: "0 btags", 1: "btags"}.items():
        for mass_type, mass_label in {"raw": "raw", "corr": "corr", "cons": "cons"}.items():
            mc_s, data_ss_s, data_ss_r_s = mc[var], data_ss[var], data_ss_r[var]
            mc_s = (
                mc_s[:, "ee" + cat, bcat, "nom", mass_type, :] +
                mc_s[:, "mm" + cat, bcat, "nom", mass_type, :]
            )
            data_ss_s = (
                data_ss_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_ss_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            data_ss_r_s = (
                data_ss_r_s[:, "ee" + cat, bcat, "none", mass_type, :] + 
                data_ss_r_s[:, "mm" + cat, bcat, "none", mass_type, :]
            )
            var_labels = {
                "corr": r"$m_{ll\tau\tau}^\mathrm{corr}$ [GeV]",
                "cons": r"$m_{ll\tau\tau}^\mathrm{cons}$ [GeV]",
                "raw": r"$m_{ll\tau\tau}^\mathrm{vis}$ [GeV]",
            }
            
            name = f"{cat}_{bcat}_{var}_{mass_label}_SS_{year_label}"
            plot_data_vs_mc(
                data_ss_s, mc_s, "mass", 
                cat_label=cat_label, var_label=var_labels[mass_type], btag_label=btag_label,
                outfile=join(outdir, name+".pdf") if plot else None,
                logscale=False,
                year=years[0],
                lumi=lumi,
                #ylim=[0.01, 30],
                data_ss=data_ss_r_s,
                blind=False,
                blind_range=None,
                rootfile=join("for_alexei/high-stats-SS-redo/reducible_validation_in_SS_region", name+".root"),
                sign="SS",
           )

In [ ]:
#########
## mll ##
#########

var = "mll"
for cat, cat_label in cat_labels.items():
    for bcat, btag_label in [("all", "all-btags")]: #{0: "0-btags", 1: "btags"}.items():
        mc_s, data_ss_s, data_ss_r_s = mc[var], data_ss[var], data_ss_r[var]
        mc_s = (
            mc_s[:, "ee" + cat, ::sum, "nom", :] + 
            mc_s[:, "mm" + cat, ::sum, "nom", :]
        )
        data_ss_s = (
            data_ss_s[:, "ee" + cat, ::sum, "none", :] + 
            data_ss_s[:, "mm" + cat, ::sum, "none", :]
        )
        data_ss_r_s = (
            data_ss_r_s[:, "ee" + cat, ::sum, "none", :] + 
            data_ss_r_s[:, "mm" + cat, ::sum, "none", :]
        )
        plot_data_vs_mc(
            data_ss_s, 
            mc_s, 
            "mll", cat_label=cat_label, var_label=r"$m_{ll}$ [GeV]", btag_label=btag_label,
            outfile=join(outdir, f"{cat}_{bcat}_{var}_SS.pdf") if plot else None,
            lumi=lumi,
            year=None,
            data_ss=data_ss_r_s,
            blind=False,
            blind_range=None,
            sign="SS",
        )

In [ ]:
import uproot
uproot.open("for_alexei/tt_0_m4l_cons_OS.root").keys()
uproot.open("for_alexei/tt_0_m4l_cons_OS.root")["ss_relaxed"].to_hist().plot1d()

In [ ]:
import uproot
uproot.open("for_alexei/tt_0_m4l_cons_SS.root").keys()
uproot.open("for_alexei/tt_0_m4l_cons_SS.root")["ss_relaxed"].to_hist().plot1d()

In [ ]:
met_phi = sum(util.load(f"../output_prompt/MC_UL_2018_all_OS.coffea")["met_phi"].values())
met_phi[::sum, ::sum, ::sum, "nom", :]

In [ ]:
met_phi = sum(util.load(f"../output_prompt_met-phi/MC_UL_2018_None_OS.coffea")["met_phi"].values())
met_phi[::sum, ::sum, ::sum, "nom", :]